<a href="https://colab.research.google.com/github/Jumper15/pytorch-work/blob/main/self_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# torch.manual_seed(6767)
# tril = torch.tril(torch.ones(3, 3))
# m = torch.rand((2,3))
# t = m.T
# # sum = tril @ t
# sum = m @ tril
# m, tril, sum


In [ ]:
# torch.manual_seed(1337)
# B, T, C = 4, 8, 1
# x = torch.randn(B, T, C)
# tril = torch.tril(torch.ones(4, 8, 8))
# m = x[0].T @ tril[0]
# tril[0], x[0].view(-1), m

In [ ]:
# tril = torch.tril(torch.ones(3,4))
# tril = tril / torch.sum(tril, dim=1, keepdims=True)
# tril

In [ ]:
# key, query, value
# n_dims = 16
# B, T, C = 4, 8, 2

import torch
import torch.nn as nn
torch.manual_seed(676767)

B, T, C = 4, 8, 2
embed_dims = 32

# for one batch
input = torch.randint(8, (B, T, C)) # TC: 4, 8, 2
Embed = nn.Embedding(8, embed_dims) # embeds: 4, 8, 2, 32
# input, Embed(input)
embed = Embed(input)
embed = embed.view(B, T, -1)
ln1 = nn.Linear(embed_dims*C, C)
logits = ln1(embed)

input.shape, embed.shape, logits.shape


(torch.Size([4, 8, 2]), torch.Size([4, 8, 64]), torch.Size([4, 8, 2]))

In [ ]:
torch.manual_seed(676767)

# attention block
head_size = 16

# value_layer = nn.Linear(C, head_size)
key_layer = nn.Linear(C, head_size)
query_layer = nn.Linear(C, head_size)

keys = key_layer(logits)
querys = query_layer(logits)
attention_block = keys @ querys.transpose(-1, -2)
attention_block.shape, attention_block[0]


(torch.Size([4, 8, 8]),
 tensor([[ 0.2850,  0.7048,  0.6008,  0.2850,  0.5184,  1.0615,  0.5184,  0.5212],
         [ 0.8059,  1.3491,  0.5115,  0.8059,  0.3369,  1.8481,  0.3369,  0.0242],
         [ 0.2421,  0.6112, -0.3186,  0.2421, -0.4721,  0.9696, -0.4721, -0.8469],
         [ 0.2850,  0.7048,  0.6008,  0.2850,  0.5184,  1.0615,  0.5184,  0.5212],
         [ 0.0979,  0.4289, -0.3835,  0.0979, -0.5192,  0.7491, -0.5192, -0.8456],
         [ 1.2717,  1.9274,  0.4811,  1.2717,  0.2283,  2.5531,  0.2283, -0.3450],
         [ 0.0979,  0.4289, -0.3835,  0.0979, -0.5192,  0.7491, -0.5192, -0.8456],
         [-0.0943,  0.1730, -0.7676, -0.0943, -0.9046,  0.4467, -0.9046, -1.2963]],
        grad_fn=<SelectBackward0>))

In [ ]:
torch.manual_seed(676767)

# tril
sfmx = nn.Softmax(dim=2)
masked_tril = attention_block.masked_fill(torch.tril(torch.ones(4, 8, 8)) == 0, float('-inf'))
output = sfmx(masked_tril)
attention_block[0], masked_tril[0], output[0]


(tensor([[ 0.2850,  0.7048,  0.6008,  0.2850,  0.5184,  1.0615,  0.5184,  0.5212],
         [ 0.8059,  1.3491,  0.5115,  0.8059,  0.3369,  1.8481,  0.3369,  0.0242],
         [ 0.2421,  0.6112, -0.3186,  0.2421, -0.4721,  0.9696, -0.4721, -0.8469],
         [ 0.2850,  0.7048,  0.6008,  0.2850,  0.5184,  1.0615,  0.5184,  0.5212],
         [ 0.0979,  0.4289, -0.3835,  0.0979, -0.5192,  0.7491, -0.5192, -0.8456],
         [ 1.2717,  1.9274,  0.4811,  1.2717,  0.2283,  2.5531,  0.2283, -0.3450],
         [ 0.0979,  0.4289, -0.3835,  0.0979, -0.5192,  0.7491, -0.5192, -0.8456],
         [-0.0943,  0.1730, -0.7676, -0.0943, -0.9046,  0.4467, -0.9046, -1.2963]],
        grad_fn=<SelectBackward0>),
 tensor([[ 0.2850,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
         [ 0.8059,  1.3491,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
         [ 0.2421,  0.6112, -0.3186,    -inf,    -inf,    -inf,    -inf,    -inf],
         [ 0.2850,  0.7048,  0.6008,  0.2850,    -

In [3]:
import torch
import torch.nn as nn
torch.manual_seed(676767)

class Attention_Head(nn.Module):
  def __init__(self, head_size=16, embed_dims=32, B=4, T=8, C=2):
    super().__init__()
    self.head_size = head_size
    self.embed_dims = embed_dims
    self.Embed = nn.Embedding(64, embed_dims)
    self.value_layer = nn.Linear(C, self.head_size)
    self.key_layer = nn.Linear(C, self.head_size)
    self.query_layer = nn.Linear(C, self.head_size)
    self.ln1 = nn.Linear(self.embed_dims*C, C)

  def masked_tril(self, logits):
    sfmx = nn.Softmax(dim=2)
    masked_tril = logits.masked_fill(torch.tril(torch.ones(4, 8, 8)) == 0, float('-inf'))
    output = sfmx(masked_tril)
    return output

  def self_attention(self, input):

    # input matrix embedding
    embed = self.Embed(input)
    embed = embed.view(B, T, -1)
    logits = self.ln1(embed)

    # attention block
    keys = self.key_layer(logits)
    querys = self.query_layer(logits)
    values = self.value_layer(logits)
    attention_block = keys @ querys.transpose(-1, -2)

    output = self.masked_tril(attention_block)
    output = output @ values
    print(output)
    return output

    # tril
    # sfmx = nn.Softmax(dim=2)
    # masked_tril = attention_block.masked_fill(torch.tril(torch.ones(4, 8, 8)) == 0, float('-inf'))
    # output = sfmx(masked_tril)
    # attention_block[0], masked_tril[0], output[0]


single_head = Attention_Head()
B, T, C = 4, 8, 2
input = torch.randint(8, (B, T, C)) # TC: 4, 8, 2

output = single_head.self_attention(input)
output.shape




tensor([[[ 4.9691e-01,  6.5349e-01, -1.3440e+00,  2.0494e-01,  5.4951e-01,
          -7.2337e-01, -3.6584e-03, -9.5597e-01, -5.8277e-01, -8.8173e-02,
          -1.1868e+00, -1.6801e-01, -7.6845e-01, -6.6694e-01,  3.9907e-01,
           1.0695e+00],
         [ 5.2738e-01,  4.4679e-01, -1.1983e+00,  1.8514e-01,  3.8327e-01,
          -6.4634e-01,  8.9988e-02, -8.7577e-01, -4.8380e-01, -1.0956e-02,
          -1.0179e+00, -2.5337e-01, -7.2716e-01, -5.8742e-01,  3.9445e-01,
           9.3410e-01],
         [ 7.1315e-01,  1.0322e-01, -9.0516e-01,  1.5128e-02,  8.8389e-02,
          -5.6959e-01,  2.9841e-01, -6.7618e-01, -3.5978e-01,  8.5501e-02,
          -7.4111e-01, -4.3633e-01, -7.6925e-01, -4.6017e-01,  5.1574e-01,
           7.4633e-01],
         [ 4.6912e-01,  3.7501e-01, -1.1737e+00,  2.4812e-01,  3.3501e-01,
          -5.9347e-01,  9.5622e-02, -8.8167e-01, -4.2881e-01,  3.2106e-02,
          -9.5728e-01, -2.6208e-01, -6.5640e-01, -5.5730e-01,  3.2713e-01,
           8.6813e-01],
    

torch.Size([4, 8, 16])

In [6]:
import torch.nn as nn

class Multi_Head_Attention():
  def __init__(self, head_num=4):
    self.head_num = head_num
    self.multi_head = nn.ModuleList([Attention_Head() for i in range(head_num)])


mha = Multi_Head_Attention()
print(mha.multi_head)

ModuleList(
  (0-3): 4 x Attention_Head(
    (Embed): Embedding(64, 32)
    (value_layer): Linear(in_features=2, out_features=16, bias=True)
    (key_layer): Linear(in_features=2, out_features=16, bias=True)
    (query_layer): Linear(in_features=2, out_features=16, bias=True)
    (ln1): Linear(in_features=64, out_features=2, bias=True)
  )
)
